# Assignmnet 1 (100 points)

**Name:** Dev Bhanushali<br>
**Email:** deb3962@thi.de<br>
**Group:** A/B <br>
**Hours spend *(optional)* :** <br>

### Question 1: Zipf’s law *(20 points)*

<p>Verify Zipf’s law on a textual corpus. The jungle book dataset is provided in the "Datasets and Resources" file. (You can also use any other dataset of your choice). <p>

<p> Provide a list of unique word sorted by their frequency in descending order. Also, give a brief discussion of the findings. You can use matplotlib library to plot the linear curve, and a log-log curve. The usage of other python modules is not permitted.</p>

imports

In [54]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import re

clean data

In [29]:
data = None
with open("./Datasets and resources/jungle_book.txt", "r") as handle:
    data = handle.read()

data = re.sub(r'[^a-zA-Z\s]', '', data)
while("\n" in data): data = data.replace("\n", " ")
data = data.split()

outliers = 0
for i in data:
    if(not(i.isalpha())):
        outliers += 1
        print(i)

find counts

In [84]:
from math import inf


freq = {}

for i in data:
    if(i in freq.keys()):
        freq[i] = freq[i] + 1
    else:
        freq[i] = 1


# find frequencies and ranks of words
values = list(freq.values())
values.sort(reverse=True)
ranks = np.array(range(len(values)))

# log10 transformation to both rank and values
log_values = np.log10(values)
log_ranks = np.log10(ranks)

reference_line_x = np.array([0, round(log_ranks[-1], 1)])
reference_line_y = np.array([round(log_values[0], 1), round(log_values[-1], 1)])

# fit a regression line and check if its slope is close to -1 to verify zipfs law
model = LinearRegression().fit(reference_line_x.reshape(-1, 1), reference_line_y)
y_pred = model.predict(log_ranks.reshape(-1, 1))

plt.plot(log_ranks, y_pred)
plt.plot()

C:\Users\devbh\AppData\Local\Temp\ipykernel_12908\1735198664.py:20: RuntimeWarning: divide by zero encountered in log10
  log_ranks = np.log10(ranks)


ValueError: Input X contains infinity or a value too large for dtype('float64').

In [81]:
reference_line_x, reference_line_y

(array([-inf,  3.8]), array([3.5, 0. ]))

### Question 2: Mutual Information *(30 points)*

<p>Pointwise mutual information quantifies the statistical dependence between events ${x_{t} = w_{1} }$ and ${ x_{t+1} = w_{2}}$. ${C(w)}$ is the absolute frequency and ${N}$ is the size of the corpus. If the probability of the next word in the corpus (${w_{2}}$) is affected by the probability of the previous word (${w_{1}}$), then ${pmi(w_{1},w_{2}) = 0}$; else the pmi value is either positive or negative. </p>

$$ pmi(w_{1},w_{2}) = log \frac{P(x_{t} = w_{1}, x_{t+1} = w_{2})} {P(x_{t} = w_{1}) . P(x_{t+1} = w_{2})} \approx log \frac {C(w_{1}w_{2}) . N} {C(w_{1}) . C(w_{2})}  $$

<p>Calculate the Pointwise mutual information (PMI) for all successive word pairs (w1, w2) in the jungle book corpus. Words (not word pairs) that occur in the corpus less than 10 times should be ignored. List the 30 word pairs with the highest pmi value and the 30 word pairs with the lowest pmi value. Document your observations and discuss the validity of the independence assumption for unigram models. If needed, you can use nltk library. </p>

### Question 3: Wikipedia language model *(50 points)*

<p> A customer of yours is dissatisfied with the quality of the speech recognition. After a conversation with the customer, you find out that he dictates books. The initial investigations on a book excerpt have shown that the language model used is not suitable. For building a better language model for the application, you have asked your customer to provide a text from the book (see "Datasets and Resources") </p>

<p> In order to save costs and also to avoid problems with copyrights, your company has decided not to use existing solutions for this project. So, you have to implement a 2-gram language model in Python from scratch. Include a short description of the data preprocessing steps, method, experiment design, hyper-parameters, and evaluation metric. Also, document your findings, drawbacks, and potential improvements. Finally, compare the performance of the language model with and without Backoff.</p>

<p> You cannot use any Python libraries like NLTK. If you need to use special Python modules, you can discuss with your instructor before submission. </p>

<h4> Datasets and Resources </h4> 

* WikiText-2 (raw/unprocessed), Train, Dev, Test

### Additional Experiments *(5 additional points - <span style="color: red;">Optional</span>)*